# Creating a dosimetry ACE file using ACEtk

In this example, we will create a dosimetry ACE file (also known as a type 'y' file) for a single nuclide using only ENDFtk and ACEtk. The dosimetry ACE file is a very simple ACE file containing a list of reaction numbers and associated cross section data sets. Contrary to the continuous energy table upon which it is based, the dosimetry file cross section data is not given on a unionised grid and the energy grid of each cross section stored in the file may be different.

For the purpose of this example, we assume that you have used NJOY2016 to generate a linearised PENDF file. This PENDF file contains the MF3 and MF10 data for a single nuclide. The first thing we will need to know is what the name of the PENDF file is, and which nuclide we're looking at.

In [ ]:
filename = 'Pu239.pendf'
z = 94
a = 239
temperature = 2.53e-8
zaid = '94239.00y'
title = 'a dosimetry file for pu239 at 293.6 K'
date = '10/29/21'
material = ''

We will now open the PENDF file using ENDFtk and loop over the MF3 and MF10 data and generate the data to be included in the ACE file using components from ACEtk. We will need to ensure that the order of the MT numbers is respected between the list of reaction numbers and the list of cross section data sets.

In [ ]:
import ENDFtk
import ACEtk

pendf = ENDFtk.tree.Tape.from_file( filename )
mf3 = pendf.materials.front().file( 3 ).parse()

# this is the list of MT numbers found in the PENDF file
numbers = []
# this is the corresponding list of DosimetryCrossSectionData objects
xs = []

# loop over each section and extract what we need
for section in mf3.sections :
    
    print( 'processing reaction number:', section.MT )
    numbers.append( section.MT )
    xs.append( ACEtk.dosimetry.CrossSectionData( energies = section.energies.to_list(),
                                                 xs = section.cross_sections.to_list() ) )

At this point, we now have a ```numbers``` and ```xs``` list that we need to create the ACE file. We still need to create the header before we can make the ACE file

In [ ]:
# get the atomic weight ratio from the last section we read
awr = section.AWR

# create a header
header = ACEtk.Header( zaid, awr, temperature, date, title, material )

# create the blocks we need for the ACE file
mtr = ACEtk.ReactionNumberBlock( numbers )
sigp = ACEtk.dosimetry.CrossSectionBlock( xs )

# create the file and write it to disk
ace = ACEtk.DosimetryTable( z, a, header, mtr, sigp )
ace.to_file( 'pu239.ace.00y' )